<a href="https://colab.research.google.com/github/jangelcode/super-duper/blob/main/machinelearning/Income_guessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import os
import io
import pandas as pd
import scipy.sparse
import pickle
import xgboost as xgb

In [ ]:
data = pd.read_csv('adult_ml.csv')
data2 = pd.DataFrame(data)
data.dtypes
data

In [ ]:
data = pd.get_dummies(data,columns=["Workclass", "Education", 'Marital_Status'], 
                      prefix=["Workclass", "Education", 'Marital_Status']).head()
data = pd.get_dummies(data,columns=["Relationship", 'native_country', 'Occupation', 'Race'], 
                      prefix=["Relationship", 'country', 'Occupation', 'Race']).head()

In [ ]:
data["Income"] = np.where(data["Income"].str.contains(">50K"), 1,0)
data["Sex"] = np.where(data["Sex"].str.contains("Male"), 1,0)

In [ ]:
data.dtypes

In [ ]:
X, y = data.iloc[:,:-1],data.iloc[:,7]
X
y

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=1)

In [ ]:
dtrain = xgb.DMatrix(X_train, y_train)
dtest = xgb.DMatrix(X_test, y_test)
dvalid = xgb.DMatrix(X_val, y_val)

In [ ]:
param = {'max_depth':4, 'eta':1, 'objective':'binary:logistic'}

In [ ]:
watchlist = [(dvalid, 'eval'), (dtrain, 'train')]
num_round = 2
bst = xgb.train(param, dtrain, num_round, watchlist)

In [ ]:
preds = bst.predict(dtest)
labels = dtest.get_label()
print('error=%f' % (sum(1 for i in range(len(preds)) if int(preds[i] > 0.5) != labels[i]) / float(len(preds))))
bst.save_model('0001.model')